In [ ]:
#for building the pyomo model
import pyomo.environ as pe

In [2]:
#for solving the pyomo model
import pyomo.opt as po

### Create the model

In [3]:
model = pe.ConcreteModel()

#### Sets
$i$: products {'A', 'B', 'C'}

$j$: factories {'F1', 'F2', 'F3', 'F4', 'F5', 'F6'}

In [4]:
model.product = pe.Set(initialize=['A', 'B', 'C'])
model.factory = pe.Set(initialize=['F1', 'F2', 'F3', 'F4', 'F5', 'F6'])

#### Parameters
$C_{ij}$: unitary cost of product $i$ in factory $j$ [€/unit] 


In [5]:
#unitary cost of product in each factory
product_costs_dict = {
    ('A', 'F1'): 25, ('A', 'F2'): 30, ('A', 'F3'): 26, ('A', 'F4'): 34, ('A', 'F5'): 32, ('A', 'F6'): 30,
    ('B', 'F1'): 30, ('B', 'F2'): 32, ('B', 'F3'): 34, ('B', 'F4'): 35, ('B', 'F5'): 38, ('B', 'F6'): 40,
    ('C', 'F1'): 40, ('C', 'F2'): 46, ('C', 'F3'): 42, ('C', 'F4'): 37, ('C', 'F5'): 40, ('C', 'F6'): 50
}


model.product_cost_in_factory = pe.Param (model.product, model.factory, initialize = product_costs_dict)



$Q_j$: capcaity of factory $j$ [unit] <br>

In [6]:

#product capacity
capacities_dict = {
    "F1": 550,
    "F2": 700,
    "F3": 1100,
    "F4": 350,
    "F5": 400,
    "F6": 450
}

model.capacity_factory = pe.Param(model.factory, initialize = capacities_dict)


$SP_i$: selling price of product $i$ [€]

In [7]:
product_selling_price_dict = {
    'A' : 60,
    'B' : 82.5,
    'C' : 108
}
model.product_selling_price = pe.Param(model.product, initialize = product_selling_price_dict)


$SOLD_i$: contract of product $i$ [unit]


In [8]:
sold_products_dict = {
    'A' : 700,
    'B' : 500,
    'C' : 600
}
model.sold_products = pe.Param(model.product, initialize = sold_products_dict)

#### Variables

$q_{ij}$: quantity of product $i$ in factory $j$ [unit]

In [ ]:
model.product_quantity_per_factory = pe.Var(model.product,model.factory, within = pe.NonNegativeReals)

#### Objective Function

max $\sum_{ij}(SP_i *q_{ij} ) - \sum_{ij}(C_{ij} *q_{ij} )$

In [17]:
def obj_rule(model):
    return sum(((model.product_selling_price[i] * model.product_quantity_per_factory[i,j]) - (model.product_cost_in_factory[i,j]*model.product_quantity_per_factory[i,j]))
                for i in model.product 
                for j in model.factory )

model.cost = pe.Objective(rule = obj_rule, sense = pe.maximize)


'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.objective.AbstractScalarObjective'>).
This is usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


#### Constraints

Constraint #1, maximum capacity of factory $j$ [unit]
$\sum_iq_{ij} \le Q_j \quad \forall j$

In [ ]:
model.max_capacity = pe.ConstraintList()

for j in model.factory:
  model.max_capacity.add(
    sum(model.product_quantity_per_factory[i,j] for f in model.food_type) >= model.nutrient_need[n]
    )